# Build a Voker Chain

This notebook provides a template for building custom Voker Chains using Wildcard integrations.

You can use this notebook to:
- Set up an email automation workflow
- Modify the chain's behavior by adjusting prompts and logic
- Create your own custom chains for different email automation needs
- Test and iterate on your automated email workflows

## Setup

### Install Dependencies

In [ ]:
%pip install -r voker_service/requirements.txt
%pip install -i https://test.pypi.org/simple/ wildcard-openai==0.0.23 --extra-index-url https://pypi.org/simple/

### Add OpenAI API Key

In [7]:
import os
os.environ["OPENAI_API_KEY"] = "" # PASTE YOUR OPENAI API KEY HERE

In [8]:
import sys
sys.path.append(os.path.abspath("."))

## Authentication Credentials
1. Go to https://wildcard-voker.vercel.app

2. Click on "Connect"

3. Use the demo account credentials. Username: logan.midchainsolutions@gmail.com Password: DemoPassword1!!

4. _Note: Connecting accounts only works with Gmail accounts approved to our Google App. If there's a specific Gmail account you want to use, please reach out to us._

5. Copy response and set it to `output` below

In [16]:
import auth_config
import json
output = """{"message":"Token received successfully.","data":{"api_service":"gmail","token_type":"Bearer","access_token":"ya29.a0ARW5m76hyRB25rV-GQy62NedeTAvGFMhOPkYRmyNSD6kUbqhLDfvzRyWobBRLzd2XcC45_fDTNttJARBvFR2KQ_D0bulPqMVSfy-2Xu2isze6_10bfPQk_fPn94cuL35-NbQ7AHwUVwL38P_ODVMHNJpkbR-1xqClBCQiHcUSYk5QBSJyGSmofi8qd-q6g1f3JuUaCgYKAeoSARISFQHGX2Mi3wO7EKrXiE6pJk0n0xMQ0A0203","scope":["https://www.googleapis.com/auth/gmail.modify","https://www.googleapis.com/auth/gmail.addons.current.message.readonly","https://www.googleapis.com/auth/gmail.readonly","https://mail.google.com/","https://www.googleapis.com/auth/gmail.settings.sharing","https://www.googleapis.com/auth/gmail.labels","https://www.googleapis.com/auth/gmail.insert","https://www.googleapis.com/auth/gmail.addons.current.action.compose","https://www.googleapis.com/auth/gmail.compose","https://www.googleapis.com/auth/gmail.send","https://www.googleapis.com/auth/gmail.settings.basic","https://www.googleapis.com/auth/gmail.addons.current.message.action"],"expires_at":1735271627,"refresh_token":null}}"""

In [17]:
output = json.loads(output)
auth_config = auth_config.AuthConfig(output["data"]).get_gmail_auth_config()

## Voker Chain

### Define Voker Inputs
Define variables that can be injected into vokers as `@<variable_name>`


In [18]:
voker_inputs = {
    "email_address": "logan.midchainsolutions@gmail.com",
    "name": "Logan",
    "order_number": "832493284"
}

### Create a system prompt
This is used to give common context to each of the vokers. It can be an empty string if you don't need it.

In [19]:
voker_system_prompt = """The users name is @name. His email is @email_address.
Perform the action specified by the user."""

### Define Voker Chain
Specify the tool actions and the prompts for each voker in the chain.


You can see a full list of available Gmail Actions here: https://docs.wild-card.ai/docs/gmail-actions

In [22]:
from wildcard_core.models import Action

# A list of vokers that will be run in sequence
voker_chain = [
    {
        "message": """Search for the messages in inbox that are about the order number @order_number. 
        I should have received a few emails.
        You should only return the thread ids.
        """,
        "tool_name": Action.Gmail.THREADS_LIST,
    },
    {
        "message": """You have been given a set of message ids. 
        Get one of the emails and understand the proposal and the vendor's price.""",
        "tool_name": Action.Gmail.THREADS_GET,

    },
    {
        "message": """You have been given a set of thread ids. 
         Get one of the emails and understand the proposal and the vendor's price.""",
        "tool_name": Action.Gmail.MESSAGES_GET,
    },
    {
        "message": """Analyze the email that you have received.
        Create a draft that is a reply to the corresponding email. Include the threadId when you are creating the draft. 
        In the draft, negotiate the price, delivery time, and unit size of the proposal from the vendor.
        """,
        "tool_name": Action.Gmail.DRAFTS_CREATE,
    }
]

### Run Voker Chain

In [ ]:
from voker_service.wildcard_node import run_single_voker

messages = []
for voker in voker_chain:
    messages.append({"role": "user", "content": voker["message"]})
    messages = await run_single_voker(voker, voker_system_prompt, auth_config, messages, voker_inputs)

## Deterministic Integrations
Use this when you DON'T want to use function calling and want to run an integration that has fixed inputs.

**See `Auth Credentials` to setup authentication before running.**



In [ ]:
from wildcard_core.models import Action
from voker_service.wildcard_node import init_tool_node

# Select the tool to run - We've selected the tool to list messages
tool_name = Action.Gmail.MESSAGES_LIST

# Initialize the tool node
tool_client, _ = await init_tool_node(tool_name, auth_config, "")

# Run the tool with arguments
arguments = {
    "userId": "me",
    "q": "from:priya.globalroute@gmail.com" # List messages with query - from Priya
}
    
await tool_client.run_tool_with_args(tool_name, **arguments)

### Hybrid Chains — Deterministic Integrations within a Voker Chain
This example shows how to include deterministic integrations in a Voker Chain.

In [26]:
# Define deterministic integration
list_emails_step = {
    "tool_name": Action.Gmail.MESSAGES_LIST,
    "arguments": {
        "userId": "me",
        "q": "from:priya.globalroute@gmail.com"
    }
}

In [27]:
voker_step_1 = {
    "message": """You've been given a list of emails. 
    Get the content of one of the emails. 
    And create a message that negotiates the price, delivery time, and unit size of the proposal from the vendor.
    """,
    "tool_name": Action.Gmail.MESSAGES_GET,
}

In [28]:
voker_step_2 = {
    "message": """You've been given the content of an email. 
    Create a draft that is a reply to the corresponding email. Include the threadId when you are creating the draft.
    """,
    "tool_name": Action.Gmail.DRAFTS_CREATE,
}

In [29]:
steps = [list_emails_step, voker_step_1, voker_step_2]

#### Run Hybrid Chain

In [ ]:
messages = []
for step in steps:
    if "message" in step:
        messages.append({"role": "user", "content": step["message"]})
        messages = await run_single_voker(step, voker_system_prompt, auth_config, messages)
    else:
        tool_response = await tool_client.run_tool_with_args(step["tool_name"], **step["arguments"])
        messages.append({"role": "user", "content": json.dumps(tool_response)})
    
    print(messages[-1]) # Print the last message